In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os


In [50]:
dataset = "dermamnist_epoch_50"
test_dir = f"./runs/{dataset}/test"

df = pd.DataFrame(columns=["seed", "experiment_type", "num_labeled", "test_acc", "test_loss", "test_ece"])

for i in range(5):
    seed_dir = f"{test_dir}/seed_{i+1}"
    for folder in os.listdir(seed_dir):
        exp_type, num_labeled = folder.split("_")

        test_metrics = pd.read_csv(f"{seed_dir}/{folder}/test_metrics.csv")

        test_acc = test_metrics["test_acc"].values[-1]
        test_loss = test_metrics["test_loss"].values[-1]
        test_ece = test_metrics["test_ece"].values[-1]

        df.loc[len(df)] = {
                "seed": i + 1,
                "experiment_type": exp_type,
                "num_labeled": int(num_labeled),
                "test_acc": test_acc,
                "test_loss": test_loss,
                "test_ece": test_ece }
         
        df = df.sort_values(by=["seed","experiment_type", "num_labeled"])

In [59]:
df.to_csv(f"{test_dir}/test_metrics.csv", index=False)

In [62]:
import matplotlib.pyplot as plt

# Assuming 'results' is your DataFrame
# Sort the DataFrame by 'num_labeled' for plotting
results = df.sort_values(by=["num_labeled"])

# Plotting
plt.figure(figsize=(10, 6), dpi=300)

# Extract unique seeds
seeds = results["seed"].unique()

# Plot accuracy vs. num_labeled for each seed
for seed in seeds:
    seed_data = results[(results["experiment_type"] == "active") & (results["seed"] == seed)]
    print(seed_data)
    # plt.errorbar(
    #     x=seed_data["num_labeled"],
    #     y=seed_data["test_loss"],
    #     yerr=seed_data["test_loss"].std(),
    #     fmt="o-",
    #     label=f"Seed {seed}",
    #     capsize=5,
    # )

# plt.xlabel("Number of Labeled Samples")
# plt.ylabel("Accuracy")
# plt.legend(loc="lower right", bbox_to_anchor=(1.25, 0.5), title="Seed")
# plt.title("Accuracy vs. Number of Labeled Samples for Active (per Seed)")

# plt.show()

    seed experiment_type  num_labeled  test_acc  test_loss  test_ece
12     1          active          500  0.694264   0.841732  0.028493
22     1          active         1000  0.718204   0.769508  0.015913
6      1          active         1500  0.708229   0.734649  0.015354
4      1          active         2000  0.729676   0.732773  0.021971
1      1          active         2500  0.726185   0.711094  0.017747
29     1          active         3000  0.724688   0.715869  0.027592
17     1          active         3500  0.726683   0.703735  0.019277
24     1          active         4000  0.730673   0.727668  0.038890
13     1          active         4500  0.720200   0.729089  0.047547
28     1          active         5000  0.732170   0.719272  0.029886
5      1          active         5500  0.734663   0.698249  0.028873
0      1          active         6000  0.739651   0.698634  0.026902
3      1          active         6500  0.726185   0.696448  0.029595
10     1          active         7

<Figure size 3000x1800 with 0 Axes>

In [55]:
import matplotlib.pyplot as plt

# Assuming 'results' is your DataFrame
# Sort the DataFrame by 'num_labeled' for plotting
results = df.sort_values(by=["num_labeled"])

# Plotting
plt.figure(figsize=(10, 6))

# Plot accuracy vs. num_labeled for active
plt.errorbar(
    x=results[results["experiment_type"] == "active"]["num_labeled"],
    y=results[results["experiment_type"] == "active"]["test_acc"],
    yerr=results[results["experiment_type"] == "active"]["test_acc"].std(),
    fmt="o-",
    label="Active",
    capsize=4,
    color="green",
)

# Plot accuracy vs. num_labeled for baseline
plt.errorbar(
    x=results[results["experiment_type"] == "baseline"]["num_labeled"],
    y=results[results["experiment_type"] == "baseline"]["test_acc"],
    yerr=results[results["experiment_type"] == "baseline"]["test_acc"].std(),
    fmt="o-",
    label="Baseline",
    capsize=4,
    color="orange",
)

plt.xlabel("Number of Labeled Samples")
plt.ylabel("Accuracy")

plt.legend(loc="lower right")
plt.grid()
plt.show()

ValueError: No axis named 1 for object type Series

<Figure size 3000x1800 with 0 Axes>

In [56]:
import matplotlib.pyplot as plt

# Assuming 'results' is your DataFrame
# Sort the DataFrame by 'num_labeled' for plotting
results = df.sort_values(by=["num_labeled"])

# Plotting
plt.figure(figsize=(10, 6), dpi=300)

# Plot accuracy vs. num_labeled for active
plt.errorbar(
    x=results[results["experiment_type"] == "active"]["num_labeled"],
    y=results[results["experiment_type"] == "active"]["test_acc"].apply(lambda x: x.mean()),  # Mean across seeds
    yerr=results[results["experiment_type"] == "active"]["test_acc"].apply(lambda x: x.std()),  # Standard deviation across seeds
    fmt="o-",
    label="Active",
    capsize=5,
    color="green",
)

# Plot accuracy vs. num_labeled for baseline
plt.errorbar(
    x=results[results["experiment_type"] == "baseline"]["num_labeled"],
    y=results[results["experiment_type"] == "baseline"]["test_acc"].apply(lambda x: x.mean()),  # Mean across seeds
    yerr=results[results["experiment_type"] == "baseline"]["test_acc"].apply(lambda x: x.std()),  # Standard deviation across seeds
    fmt="o-",
    label="Baseline",
    capsize=5,
    color="orange",
)

plt.xlabel("Number of Labeled Samples")
plt.ylabel("Accuracy")
plt.legend(loc="lower right")
plt.grid()
plt.show()

AttributeError: 'float' object has no attribute 'mean'

<Figure size 3000x1800 with 0 Axes>